In [1]:
# Various setup, probably mostly unused

from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
from bart.sched.SchedMultiAssert import SchedMultiAssert
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
logging.getLogger('EnergyMeter').setLevel(logging.DEBUG)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy
import tests.eas.generic
from tests.eas.generic import EnergyModelTest
import numpy as np
from IPython.display import display
from wlgen import RTA, Periodic
from executor import Executor

2017-02-07 18:18:54,109 INFO    : root         : Using LISA logging configuration:
2017-02-07 18:18:54,110 INFO    : root         :   /home/brejac01/sources/lisa/logging.conf


In [2]:
# Some nonsense to get caiman to work on Brendan's computer
p = os.getenv('PATH').split(':')
caiman_path = '/opt/ds5_v5.23.0/bin'
if caiman_path not in p:
    p.insert(0, caiman_path)
    os.environ['PATH'] = ':'.join(p)

In [3]:
from platforms.juno_energy import juno_energy

In [4]:
from platforms.hikey_energy import hikey_energy

In [5]:
te = TestEnv(test_conf={
        'modules': ['cgroups'], 
        'ftrace': {
            'events': ['sched_switch', 'cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task']
        }
    }, force_new=True)

2017-02-07 18:18:54,186 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2017-02-07 18:18:54,187 INFO    : TestEnv      : Loading default (file) target configuration
2017-02-07 18:18:54,189 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2017-02-07 18:18:54,191 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-02-07 18:18:54,192 INFO    : TestEnv      : External tools using:
2017-02-07 18:18:54,193 INFO    : TestEnv      :    ANDROID_HOME: /work/android-sdk-linux
2017-02-07 18:18:54,194 INFO    : TestEnv      :    CATAPULT_HOME: /home/brejac01/sources/lisa/tools/catapult
2017-02-07 18:18:54,196 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cgroups', 'cpufreq']
2017-02-07 18:18:54,197 INFO    : TestEnv      : Connecting Android target [04D314DF000BC57F]
2017-02-07 18:18:54,199 INFO    : TestEnv      : Connection settings:
2017-02-07 18:18:54,200 INFO    : TestEnv      :    {'

In [6]:
# Some nonsense to make LISA treat HiKey with the respect it deserves
from trappy.stats.Topology import Topology
hikey_topology = Topology(clusters=[[0, 1, 2, 3], [4, 5, 6, 7]])
te.topology = hikey_topology
te.platform['clusters'] = {'big': [0, 1, 2, 3], 'little': [4, 5, 6, 7]}

In [7]:
def spread_evenly(ntasks, ncpus):
    ret = [0 for _ in range(ncpus)]
    for i in range(ntasks):
        ret[i % ncpus] += 1
    return ret

def get_various_packing_placements(ntasks, ncpus):
    max_spread = spread_evenly(ntasks, ncpus)
    
    max_pack = [ntasks] + [0] * (ncpus - 1)
    
    half_pack = [ntasks / 2] + spread_evenly(ntasks - (ntasks / 2), ncpus -1)
    
    return [max_spread, half_pack, max_pack]

In [8]:
def get_everything(nrg_model, cpu=0, task_pct=20):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    max_cap = cpu_node.max_capacity
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(max_cap / task_cap)
    ntasks = tasks_per_cpu * ncpus
    task_distribs = get_various_packing_placements(ntasks, ncpus)
    
    util_distribs = []
    wloads = []
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)

    # Stagger the workloads so that they prevent shared idle states
    period_s = 10e-3
    stagger_s = 0.7e-3
    delays = np.arange(period_s, step=stagger_s)
    
    if len(delays) > ntasks:
        print "WARNING: not enough tasks to cover period with wakeups"
    
    tasks = {}
    for i in range(ntasks):
        tasks['{}pct_{}'.format(task_pct, i)] = {
            'kind': 'Periodic',
            'params': {
                'duty_cycle_pct': task_pct,
                'period_ms' : period_s * 1e3,
                'delay_s' : delays[i % len(delays)],
                'duration_s': 2
            }
        }
    
    wloads = {
        'forced_pack': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
                'cpus': [0, 1, 2, 3]
            }
        },
        'any_cpu': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
            }
        }
    }
        
    return task_distribs, util_distribs, wloads

In [9]:
task_distribs, util_distribs, wloads = get_everything(hikey_energy)

In [10]:
poor_mans_powersave = { # No powersave governor in hikey_defconfi
    'governor': 'userspace',
    'freqs':{
        0: 208000,
    }
}
performance = {
    'governor': 'performance'
}
executor = Executor(te, {
        'confs': [{
                'tag': 'myconf',
                'flags': ['ftrace', 'freeze_userspace'],
                'cpufreq': performance,
            }],
        'wloads': wloads,
        'iterations': 5
    })

2017-02-07 18:19:35,686 INFO    : Executor     : Loading custom (inline) test configuration
2017-02-07 18:19:35,687 INFO    : Executor     : 
2017-02-07 18:19:35,688 INFO    : Executor     : ################################################################################
2017-02-07 18:19:35,689 INFO    : Executor     : Experiments configuration
2017-02-07 18:19:35,690 INFO    : Executor     : ################################################################################
2017-02-07 18:19:35,691 INFO    : Executor     : Configured to run:
2017-02-07 18:19:35,692 INFO    : Executor     :      1 target configurations:
2017-02-07 18:19:35,693 INFO    : Executor     :       myconf
2017-02-07 18:19:35,694 INFO    : Executor     :      2 workloads (5 iterations each)
2017-02-07 18:19:35,695 INFO    : Executor     :       any_cpu, forced_pack
2017-02-07 18:19:35,696 INFO    : Executor     : Total: 10 experiments
2017-02-07 18:19:35,697 INFO    : Executor     : Results will be collected under:

In [11]:
executor.run()

2017-02-07 18:19:39,323 INFO    : Executor     : 
2017-02-07 18:19:39,325 INFO    : Executor     : ################################################################################
2017-02-07 18:19:39,326 INFO    : Executor     : Experiments execution
2017-02-07 18:19:39,327 INFO    : Executor     : ################################################################################
2017-02-07 18:19:39,328 INFO    : Executor     : 
2017-02-07 18:19:39,330 INFO    : Executor     : ================================================================================
2017-02-07 18:19:39,331 INFO    : Executor     : configuring target for [myconf] experiments
2017-02-07 18:19:40,752 WARNING : Executor     : Setting target SELinux in permissive mode
2017-02-07 18:19:41,532 INFO    : Executor     : Configuring all CPUs to use [performance] cpufreq governor
2017-02-07 18:19:42,193 INFO    : Workload     : Setup new workload any_cpu
2017-02-07 18:19:42,195 INFO    : Workload     : Workload duration defi

2017-02-07 18:19:54,851 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000434
2017-02-07 18:20:00,126 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:20:00,128 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/1/trace.dat
2017-02-07 18:20:00,129 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:20:00,130 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/1/trace_stat.json
2017-02-07 18:20:00,132 INFO    : Executor     : Calling postrun step
2017-02-07 18:20:02,923 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:20:03,649 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:20:03,651 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:20:03,652 INFO    : Executor     : Experiment 1/10, [myconf:any_cpu] 2/5
2017-02-07 18:20:03,731 INFO    : Executor     : Freezing a

2017-02-07 18:20:13,038 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/any_cpu_00.json 2>&1
2017-02-07 18:20:15,512 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000444
2017-02-07 18:20:20,812 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:20:20,814 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/2/trace.dat
2017-02-07 18:20:20,815 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:20:20,816 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/2/trace_stat.json
2017-02-07 18:20:20,818 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:20:21,549 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:20:21,551 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:20:21,552 INFO    : Executor     : Experiment 2/10, [myconf:any_

2017-02-07 18:20:31,067 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/any_cpu_00.json 2>&1
2017-02-07 18:20:33,552 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000444
2017-02-07 18:20:39,018 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:20:39,020 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/3/trace.dat
2017-02-07 18:20:39,021 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:20:39,022 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/3/trace_stat.json
2017-02-07 18:20:39,023 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:20:39,756 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:20:39,758 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:20:39,759 INFO    : Executor     : Experiment 3/10, [myconf:any_

2017-02-07 18:20:49,031 INFO    : Workload     : Workload execution START:
2017-02-07 18:20:49,032 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/any_cpu_00.json 2>&1
2017-02-07 18:20:51,517 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000437
2017-02-07 18:20:56,790 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:20:56,792 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/4/trace.dat
2017-02-07 18:20:56,793 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:20:56,794 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/4/trace_stat.json
2017-02-07 18:20:56,795 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:20:57,550 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:20:57,552 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
201

...

2017-02-07 18:21:27,831 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000462
2017-02-07 18:21:32,967 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:21:32,969 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/1/trace.dat
2017-02-07 18:21:32,970 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:21:32,971 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/1/trace_stat.json
2017-02-07 18:21:32,973 INFO    : Executor     : Calling postrun step
2017-02-07 18:21:35,775 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:21:36,497 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:21:36,499 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:21:36,500 INFO    : Executor     : Experiment 6/10, [myconf:forced_pack] 2/5
2017-02-07 18:21:36,575 INFO    : Executor     

2017-02-07 18:21:45,896 INFO    : Workload     : Workload execution START:
2017-02-07 18:21:45,897 INFO    : Workload     :    /data/local/tmp/bin/taskset 0xF /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/forced_pack_00.json 2>&1
2017-02-07 18:21:48,501 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000471
2017-02-07 18:21:53,532 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:21:53,534 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/2/trace.dat
2017-02-07 18:21:53,535 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:21:53,536 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/2/trace_stat.json
2017-02-07 18:21:53,537 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:21:54,266 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:21:54,268 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

2017-02-07 18:22:06,177 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000468
2017-02-07 18:22:11,415 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:22:11,417 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/3/trace.dat
2017-02-07 18:22:11,419 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:22:11,420 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/3/trace_stat.json
2017-02-07 18:22:11,421 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:22:12,143 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:22:12,145 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:22:12,146 INFO    : Executor     : Experiment 8/10, [myconf:forced_pack] 4/5
2017-02-07 18:22:12,217 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine

2017-02-07 18:22:21,579 INFO    : Workload     :    /data/local/tmp/bin/taskset 0xF /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/forced_pack_00.json 2>&1
2017-02-07 18:22:24,222 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000471
2017-02-07 18:22:29,393 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:22:29,395 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/4/trace.dat
2017-02-07 18:22:29,397 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:22:29,398 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/4/trace_stat.json
2017-02-07 18:22:29,399 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:22:30,124 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:22:30,126 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-07 18:22:30,127 INFO    : 

2017-02-07 18:22:42,173 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000470
2017-02-07 18:22:47,153 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 18:22:47,156 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/5/trace.dat
2017-02-07 18:22:47,157 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 18:22:47,158 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/5/trace_stat.json
2017-02-07 18:22:47,159 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 18:22:47,881 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 18:22:47,883 INFO    : Executor     : Restoring target SELinux mode: Enforcing

2017-02-07 18:22:48,104 INFO    : Executor     : 
2017-02-07 18:22:48,106 INFO    : Executor     : ################################################################################
2017-02-07 18:22:48,107 INFO    : Executor     : Experiments execut

In [12]:
for experiment in executor.experiments:
    plot_trace(Trace(te.platform, experiment.out_dir, ['sched_switch']).ftrace)

In [13]:
def examine_experiment(experiment):
    trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    ma = SchedMultiAssert(trace.ftrace, hikey_topology, experiment.wload.tasks.keys())
    cluster_conclusions = []
    for cluster in te.topology.get_level('cluster'):
        residencies = [t['residency'] for p, t in ma.getResidency('cluster', cluster, percent=True).iteritems()]
        if all(r > 90 for r in residencies):
            cluster_conclusions.append('PACKED')
        else:
            cluster_conclusions.append(None)
    packed = [i for i, c in enumerate(cluster_conclusions) if c == 'PACKED']
    with open(os.path.join(experiment.out_dir, 'energy.json')) as f:
        energy = json.load(f)
        energy = energy['BAT']
    if not packed:
        print "I don't think we packed onto any cluster"
        return False, energy
        # plot_trace(trace.ftrace)
    elif len(packed) == 1:
        [i] = packed
        print 'I think we packed onto cluster {} ({})'.format(i, te.topology.get_node('cluster', i))
        return True, energy
    else:
        # this code is
        print 'what is this'
        # who even.. who wrote this shit
        print 'i dont even'
        # Microsoft recommends Windows 10 with Microsoft Edge® for a more secure browsing experience
        raise Exception(RuntimeError(GenericBamboozlementComplaintFactory2().generateComplaint('bamboozled!!!')))

In [14]:
previous_energy_records = [[], []]

In [15]:
energy_records = previous_energy_records
for experiment in executor.experiments:
    packed, energy = examine_experiment(experiment)
    energy_records[packed].append(energy)

I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I think we packed onto cluster 0 ([0, 1, 2, 3])
I think we packed onto cluster 0 ([0, 1, 2, 3])
I think we packed onto cluster 0 ([0, 1, 2, 3])
I think we packed onto cluster 0 ([0, 1, 2, 3])
I think we packed onto cluster 0 ([0, 1, 2, 3])


In [16]:
import numpy as np

In [17]:
np.mean(energy_records[0]), np.std(energy_records[0])

(0.00043935848121705946, 3.8907429882254373e-06)

In [18]:
np.mean(energy_records[1]), np.std(energy_records[1])

(0.00046827775689218566, 3.5290230829524806e-06)

In [19]:
energy_records

[[0.0004342702757566875,
  0.000443813503675491,
  0.000444027552846793,
  0.0004374034523165369,
  0.0004372776214897887],
 [0.0004615823043594779,
  0.00047089765879731907,
  0.0004678687081587409,
  0.00047094071819979715,
  0.00047009939494559356]]

In [20]:
traces = [Trace(te.platform, e.out_dir, ['sched_switch', 'cpu_frequency']) for e in executor.experiments]

In [21]:
te.platform

{'clusters': {'big': [0, 1, 2, 3], 'little': [4, 5, 6, 7]},
 'cpus_count': 8,
 'freqs': {0: [208000, 432000, 729000, 960000, 1200000]},
 'nrg_model': {u'big': {u'cluster': {u'nrg_max': 112},
   u'cpu': {u'cap_max': 1024, u'nrg_max': 670}},
  u'little': {u'cluster': {u'nrg_max': 112},
   u'cpu': {u'cap_max': 1024, u'nrg_max': 670}}},
 'topology': [[0, 1, 2, 3, 4, 5, 6, 7]]}

In [22]:
freqs = [208000, 432000, 729000, 960000, 1200000]
te.platform['freqs']['big'] = freqs
te.platform['freqs']['little'] = freqs

In [23]:
trace = traces[0]
trace.analysis.frequency.plotClusterFrequencies()

2017-02-07 18:23:20,619 WARNING : Analysis     : Events [cpu_frequency] not found, plot DISABLED!


In [24]:
te.target.big_core

In [25]:
trace.analysis.tasks.plotTasks()

2017-02-07 18:23:20,635 WARNING : Analysis     : Events [sched_load_avg_task] not found, plot DISABLED!


In [26]:
# For android-hikey-4.4
energy_records_hikey_regular = [[0.0007220933633207071,
  0.0007259948168912483,
  0.00073977954186192,
  0.0007320675803349075,
  0.0007652096829866224,
  0.0007528051382064471,
  0.0007435907446651704,
  0.0006940457338712526],
 [0.0006728093519416772,
  0.0007162223819813184,
  0.000712395270479278,
  0.000673710608013742,
  0.0006976346544107044,
  0.0006925454978772343,
  0.0006657587436840893,
  0.0007349576824902922,
  0.0006929229334050849,
  0.0007159691648384068,
  0.0007202090557403527,
  0.0006954405751603721]]

In [27]:
raise Exception('HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH')

Exception: HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH

##### 💀💀 The code graveyard 💀💀
*abandon hope all ye who scroll past here, for here is the threshold of the graveyard, whither is sent the code of yore, remembered but not missed, respected but not loved, written by the hand of fearful men and trialled by the silicon of scorned machines*

In [ ]:
def get_task_distribs(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    return task_distribs

In [ ]:
def get_util_distribs(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    util_distribs = []
    
    task_cap = int(1024 * (task_pct / 100.))
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
    return util_distribs

In [ ]:
def get_wloads(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    for task_distrib in 

In [ ]:
util_distribs = get_util_distribs(hikey_energy)
for ud in util_distribs:
    display(hikey_energy.estimate_from_cpu_util(ud))

In [ ]:
hikey_energy.cpu_nodes[0].min_capacity

In [ ]:
def get_cluster_count_df(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    tasks = experiment.wload.tasks.keys()
    pids = [trace.getTaskByName(t)[0] for t in tasks]
    task_cpus = trace.data_frame.task_cpus()[pids]
    # drop consecutive duplicates
    task_cpus = task_cpus[(task_cpus.shift(+1) != task_cpus).any(axis=1)]

    df = pd.DataFrame(columns=[str(c) for c in clusters])
    
    for (time, row_in) in task_cpus.iterrows():
        cluster_task_counts = [0 for c in clusters]
        for task_cpu in row_in:
            if isnan(task_cpu):
                continue
            for i, cluster in enumerate(clusters):
                if int(task_cpu) in cluster:
                    cluster_task_counts[i] += 1
                    break
    
        df.loc[time] = pd.Series(cluster_task_counts, index=df.columns)
        
    return df

def were_tasks_cluster_packed(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    cluster_counts = get_cluster_count_df(experiment, clusters, trace)
    

In [ ]:
def get_nrg(model, util):
    nrg = model.estimate_from_cpu_util(util, zero_idle=True)
    return sum(nrg.values())